In [2]:
!pip install facenet-pytorch

  Using cached facenet_pytorch-2.6.0-py3-none-any.whl (1.9 MB)
  Using cached numpy-1.26.4-cp39-cp39-win_amd64.whl (15.8 MB)
     ------------------------------------ 198.5/198.5 MB 168.2 kB/s eta 0:00:00
  Using cached torchvision-0.17.2-cp39-cp39-win_amd64.whl (1.2 MB)
  Using cached pillow-10.2.0-cp39-cp39-win_amd64.whl (2.6 MB)
  Attempting uninstall: Pillow
    Found existing installation: Pillow 9.2.0
    Uninstalling Pillow-9.2.0:
      Successfully uninstalled Pillow-9.2.0
  Attempting uninstall: numpy
    Found existing installation: numpy 1.21.5
    Uninstalling numpy-1.21.5:
      Successfully uninstalled numpy-1.21.5
  Attempting uninstall: torch
    Found existing installation: torch 2.4.1
    Uninstalling torch-2.4.1:
      Successfully uninstalled torch-2.4.1
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.19.1
    Uninstalling torchvision-0.19.1:
      Successfully uninstalled torchvision-0.19.1


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
daal4py 2021.6.0 requires daal==2021.4.0, which is not installed.
scipy 1.9.1 requires numpy<1.25.0,>=1.18.5, but you have numpy 1.26.4 which is incompatible.
numba 0.55.1 requires numpy<1.22,>=1.18, but you have numpy 1.26.4 which is incompatible.


In [ ]:
!pip install opencv-python

In [ ]:
import os
import cv2
import time
from facenet_pytorch import MTCNN, InceptionResnetV1
import torch
from PIL import Image
import skfuzzy as fuzz
import numpy as np

# Initialize MTCNN and InceptionResnetV1
mtcnn0 = MTCNN(image_size=240, margin=0, keep_all=False, min_face_size=40)
mtcnn = MTCNN(image_size=240, margin=0, keep_all=True, min_face_size=40)
resnet = InceptionResnetV1(pretrained='vggface2').eval() 

# Initialize empty lists to store embeddings and names
embedding_list = []
name_list = []

# Process images in the photos folder
photos_folder = "photos"  # Path to the folder containing images
for filename in os.listdir(photos_folder):
    image_path = os.path.join(photos_folder, filename)
    if image_path.lower().endswith(('.png', '.jpg', '.jpeg')):
        # Process each image for embeddings
        img = Image.open(image_path)
        face, prob = mtcnn0(img, return_prob=True)

        if face is not None and prob > 0.92:
            emb = resnet(face.unsqueeze(0)) # Dimension
            embedding_list.append(emb.detach()) # Tensor
            name_list.append(filename.split('.')[0])  # Use the filename (without extension) as the name

# Save the data (embeddings and names)
data = [embedding_list, name_list]
torch.save(data, 'data.pt')  # Save data.pt file

# Fuzzy Logic Membership Functions for Distance and Confidence
def fuzzy_distance(distance):
    dist_range = np.linspace(0, 2, 100)

    # Distance fuzzy sets (Close, Medium, Far)
    close = fuzz.trimf(dist_range, [0, 0, 0.5])
    medium = fuzz.trimf(dist_range, [0, 0.5, 1.0])
    far = fuzz.trimf(dist_range, [0.5, 1.0, 2.0])
    
    # Calculate the membership values for the given distance
    close_membership = fuzz.interp_membership(dist_range, close, distance)
    medium_membership = fuzz.interp_membership(dist_range, medium, distance)
    far_membership = fuzz.interp_membership(dist_range, far, distance)
    
    return close_membership, medium_membership, far_membership

def fuzzy_confidence(confidence):
    conf_range = np.linspace(0, 1, 100)
    
    # Confidence fuzzy sets (Low, Medium, High)
    low = fuzz.trimf(conf_range, [0, 0, 0.5])
    medium = fuzz.trimf(conf_range, [0, 0.5, 1.0])
    high = fuzz.trimf(conf_range, [0.5, 1.0, 1.0])
    
    # Calculate the membership values for the given confidence
    low_membership = fuzz.interp_membership(conf_range, low, confidence)
    medium_membership = fuzz.interp_membership(conf_range, medium, confidence)
    high_membership = fuzz.interp_membership(conf_range, high, confidence)
    
    return low_membership, medium_membership, high_membership

# Load the saved data
load_data = torch.load('data.pt')
embedding_list = load_data[0]
name_list = load_data[1]

# Open the webcam
cam = cv2.VideoCapture(0)

while True:
    ret, frame = cam.read()
    if not ret:
        print("Failed to grab frame, try again")
        break

    # Convert frame to image for MTCNN processing
    img = Image.fromarray(frame)
    img_cropped_list = None  # Initialize to default
    match_probability = 0.0  # Initialize to default

    try:
        img_cropped_list, prob_list = mtcnn(img, return_prob=True)
    except Exception as e:
        print(f"Error during face detection: {e}")

    if img_cropped_list is not None:
        boxes, _ = mtcnn.detect(img)

        for i, prob in enumerate(prob_list):
            if prob > 0.90:
                emb = resnet(img_cropped_list[i].unsqueeze(0)).detach()

                # List of distances for face recognition
                dist_list = [torch.dist(emb, emb_db).item() for emb_db in embedding_list]

                min_dist = min(dist_list)  # Get minimum distance
                min_dist_idx = dist_list.index(min_dist)  # Get index of minimum distance
                name = name_list[min_dist_idx]  # Get name corresponding to the minimum distance

                # Get bounding box for detected face
                box = boxes[i]
                original_frame = frame.copy()  # Store the frame before drawing on it

                # Apply fuzzy logic to determine the matching probability
                close_membership, medium_membership, far_membership = fuzzy_distance(min_dist)
                low_membership, medium_conf_membership, high_membership = fuzzy_confidence(prob)

                # Compute weighted average of memberships
                match_probability = (close_membership * high_membership) + \
                                    (medium_membership * medium_conf_membership) + \
                                    (far_membership * low_membership)

                # Scale the match probability to be in the range of [0, 1]
                match_probability = np.clip(match_probability, 0, 1)

                # Draw the name and match probability on the frame
                frame = cv2.putText(frame, f'{name} Match Probability: {match_probability:.2f}', 
                                    (int(box[0]), int(box[1])), 
                                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)

                # Draw the bounding box around the face
                frame = cv2.rectangle(frame, (int(box[0]), int(box[1])), 
                                      (int(box[2]), int(box[3])), (0, 255, 0), 2)

    # Display the frame
    cv2.imshow("Output", frame)

    k = cv2.waitKey(1)
    if k % 256 == 27:  # ESC key to exit
        break

    # Save the image of the recognized person if match probability is high
    if match_probability > 0.8 and 'name' in locals() and not os.path.exists(f'photos/{name}'):
        os.mkdir(f'photos/{name}')

        img_name = f"photos/{name}/{int(time.time())}.jpeg"
        cv2.imwrite(img_name, original_frame)
        print(f"Saved: {img_name}")

# Release the webcam and close windows
cam.release()
cv2.destroyAllWindows()


C:\Users\Hi\anaconda3\lib\site-packages\scipy\__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
